In [1]:
import requests
from bs4 import BeautifulSoup
from functools import reduce
import pandas as pd
import re
import numpy as np
import time
from flashtext.keyword import KeywordProcessor
from natasha import DatesExtractor

In [2]:
def to_dict_1(s):
    if len(s) == 0 or s == '[]':
        return False
    s = [el.split(',') for el in s.replace(']}','').replace('[{', '').split('},{')]
    d = dict()
    for row in s:
        el = list(map(lambda x: reduce(lambda y1, y2: y1 + y2, re.findall('\d', x)), row))
        d['оценено на ' + el[0]] = int(el[1])
    return d


def to_dict_2(a):
    if len(a) == 0:
        return np.nan
    s = [el.split(',') for el in a.replace(']}','').replace('[{', '').split('},{')]
    d = dict()
    for row in s:
        p = reduce(lambda y1, y2: y1 + y2, re.findall('\d', row[1]))
        d[row[0][8:-1]] = int(p)
    return d


def to_dict_3(a):
    d = dict()
    for row in a:
        p = row.split(' ')
        if 'коммент' in p[1]:
            d['комментариев'] = p[0]
        elif 'отзыв' in p[1]:
            d['отзывов'] = p[0]
        elif 'реценз' in p[1]:
            d['рецензии'] = p[0]
    return d

def to_str(a):
    if len(a) > 0:
        return reduce(lambda x, y: x + ', ' + y, a)
    else: 
        return np.nan
    


In [10]:
%%time
start_page = 1
end_page = 576
urls = 'https://shikimori.org/animes/kind/!music/status/released/page/'
data_frame = pd.DataFrame()



for link in range(start_page, end_page):
    r1 = requests.get(urls + str(link))
    soup_pages = BeautifulSoup(r1.text, 'html5lib')
    list_pages = (x.get('data-href') for x in soup_pages.find_all('div', class_ = 'cover linkeable anime-tooltip'))
    for url in list_pages:
        time.sleep(1)
        r2 = requests.get(url)
        soup = BeautifulSoup(r2.text, 'html5lib')
        anime = dict()
        anime['название'] = soup.find('h1').text
        
        for row in zip(soup.find_all('div', class_ = 'key'), soup.find_all('div', class_ = 'value')):
            if row[0].text == 'Статус:':
                anime[row[0].text] = row[1].span.get('data-text') 
                anime['время выхода'] = row[1].text[1:]
            elif row[0].text == 'Жанры:':
                genre = to_str([tag.get('data-text') for tag in row[1].find_all('span', class_ = 'genre-ru')])
                anime['Жанр:'] = genre
            elif row[0].text == 'Альтернативные названия:':
                url_1 = soup.find('span', class_  = 'other-names click-loader').get('data-href')
                r_alternative = requests.get(url_1).text
                soup_alternative = BeautifulSoup(r_alternative, 'html5lib')
                for language in zip(soup_alternative.find_all('div', class_ = 'key'), 
                                  soup_alternative.find_all('div', class_ = 'value')):
                    anime[language[0].text] = language[1].text 
        
            else:
                anime[row[0].text] = row[1].text

        date = soup.find('span', class_ = 'b-tooltipped mobile unprocessed')
        if 'в' and 'гг.' in anime['время выхода']:
            if date:
                a1 = 'G' not in date.get('title')
                a2 = 'PG' not in date.get('title')
                a3 = 'PG-13' not in date.get('title')
                a4 = 'R' not in date.get('title')
                a5 = 'Rx' not in date.get('title')
                if a1 and a2 and a3 and a4 and a5:
                    anime['время выхода'] = date.get('title')


 
        commit = soup.find('div', class_ = 'additional-links')  
        if commit:
            p = [row.text for row in commit.find_all('span') 
                    if row.text not in set(['Косплей', 'Арт с имиджборд'])]
            anime.update(to_dict_3(p))
            

        anime['описание'] = soup.find('div', class_ = 'text').text

        rating = soup.find('div', 'text-score')
        if rating:
            p = [r.text for r in rating.find_all('div')]
            anime['оценка'] = p[0]
            anime['расшифровка оценки'] = p[1]


        anime['студия']  = to_str([name.get('title').replace('Аниме студии', '') for name in soup.find('div', class_ = 'c-info-right').find_all('a') 
                     if 'Аниме студии' in name.get('title')])
        anime['награды'] = [name.get('title') for name in soup.find('div', class_ = 'c-info-right').find_all('a') 
                     if 'Аниме студии' not in name.get('title')]
        
        anime['количество наград'] = len(anime['награды'])
        if len(anime['награды']) == 0:
            anime['награды'] = np.nan
        else:
            anime['награды'] = to_str(anime['награды'])
        
        
        rating_people = soup.find('div', id = 'rates_scores_stats')
        if rating_people:
            if to_dict_1(rating_people.get('data-stats')):
                anime.update(to_dict_1(rating_people.get('data-stats')))

    

        list_people = soup.find('div', id = 'rates_statuses_stats')
        if list_people:
            anime.update(to_dict_2(list_people.get('data-stats')))

        favoured = soup.find('div', class_ = 'b-favoured')
        if favoured:
            favoured_1 = favoured.find(class_ = 'count')
            if favoured_1:
                anime['в избранном'] = int(favoured_1.text)
            else:
                anime['в избранном'] = favoured.find_all(class_ = 'b-user c-column avatar').__len__()

        
        try:
            data = set(list(map(lambda x: x.find(class_ = 'count').text if 'Клубы' in x.text else 1, 
                 soup.find_all('div', class_ = 'subheadline m10'))))
            if len(data) > 1:
                data = data.difference(set([1]))
                anime['клубы'] = int(list(data)[0])
        except AttributeError:
            anime['клубы'] = soup.find('div', class_ = 'b-clubs one-line').find_all('div').__len__()

        
        url_2 = soup.find('div', class_='resources-loader postloaded').get('data-href')
        r = requests.get(url_2)
        soup_char = BeautifulSoup(r.text, 'html5lib')

        bound = soup_char.find('div', class_ = 'c-column block_m')
        if bound:
            anime['связанных'] = bound.find_all('div', class_ = 'b-db_entry-note').__len__()


        author = soup_char.find('div', class_ = 'c-column c-authors block_m')
        if author:
            anime['авторы'] = to_str([a.text for a in author.find_all('a')])


        heroes  = soup_char.find('div', class_ = 'cc-characters')
        if heroes:
            anime['главные герои'] = to_str(list(map(lambda x: x.text, heroes.find_all('article'))))
  
        data_frame = data_frame.append(anime, ignore_index = True)
        
        

Wall time: 9h 18min 39s


In [11]:
print(url, link) 

https://shikimori.org/animes/13455-zumomo-to-nupepe 575


In [12]:
to_index = ['Quit', 'Length of the episode', 'Other titles', 'Genre',
'Scheduled', 'Delayed', 'Revised', 'In English',
'In Japanese', 'Viewed', 'Rating', 'Look', 'Status', 'Type',
'Episodes', 'Authors',' In favorites', 'Exit time', 'Protagonists',
'The number of awards', 'Comments', 'Awards', 'Title', 'Description',
'Reviews', 'Rated at 1', 'Rated at 10', 'Rated at 2', 
'Rated at 3', 'Rated at 4', 'Rated at 5', 'Rated at 6', 
'Rated at 7', 'Rated at 8', 'Rated at 9', 'Valued',
'Decoding of evaluation', 'Connected', 'Studio', 'critique']
d = {a: b for a, b in zip(data_frame, to_index)}




a = data_frame.rename(index = str, columns = d)

In [13]:
a.to_csv('exs.csv', sep = '\t', encoding = 'utf-8')

In [ ]:
! shutdown /s /t 60

In [30]:
data = pd.read_csv('exs.csv', sep = '\t', encoding = 'utf-8', index_col = 0)
data

,Quit,Length of the episode,Other titles,Genre,Scheduled,Delayed,Revised,In English,In Japanese,Viewed,...,Rated at 5,Rated at 6,Rated at 7,Rated at 8,Rated at 9,Valued,Decoding of evaluation,Connected,Studio,critique
0,1084.0,24 мин.,"Hagane no Renkinjutsushi: Fullmetal Alchemist,...","Экшен, Военное, Приключения, Комедия, Драма, М...",12431.0,1273.0,490.0,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,38153.0,...,219.0,447.0,1303.0,3063.0,5408.0,9.25,Великолепно,2.0,5.0,Bones
1,128.0,1 час 46 мин.,NaN,"Сверхъестественное, Драма, Романтика, Школа",15627.0,287.0,129.0,Your Name.,君の名は。,42336.0,...,187.0,446.0,1378.0,3096.0,5541.0,9.22,Великолепно,3.0,2.0,CoMix Wave
2,488.0,24 мин.,Gintama' (2015),"Экшен, Комедия, Исторический, Пародия, Самураи...",4635.0,370.0,45.0,Gintama Season 4,銀魂°,6672.0,...,39.0,59.0,182.0,389.0,723.0,9.18,Великолепно,NaN,6.0,Bandai Namco Pictures
3,2026.0,24 мин.,シュタゲ,"Фантастика, Триллер",17379.0,2024.0,382.0,Steins;Gate,シュタインズ・ゲート,42760.0,...,365.0,641.0,1718.0,3789.0,6943.0,9.14,Великолепно,4.0,8.0,White Fox
4,443.0,24 мин.,Gintama (2011),"Экшен, Фантастика, Комедия, Исторический, Паро...",4422.0,315.0,46.0,Gintama Season 2,銀魂',8704.0,...,58.0,90.0,284.0,634.0,1131.0,9.13,Великолепно,NaN,5.0,Sunrise
5,726.0,23 мин.,HxH (2011),"Экшен, Приключения, Супер сила, Сёнен",7148.0,794.0,269.0,Hunter x Hunter,HUNTER×HUNTER ハンターハンター,14949.0,...,98.0,202.0,685.0,1578.0,2362.0,9.12,Великолепно,NaN,7.0,Madhouse Studios
6,238.0,26 мин.,"LoGH, LotGH, Gin'eiden, GinEiDen, Heldensagen ...","Военное, Фантастика, Космос, Драма",3872.0,286.0,12.0,Legend of the Galactic Heroes,銀河英雄伝説,1065.0,...,18.0,30.0,70.0,130.0,168.0,9.10,Великолепно,1.0,7.0,"Artland, Magic Bus"
7,294.0,24 мин.,"Gintama' (2012), Gintama' Overdrive, Kintama","Экшен, Комедия, Исторический, Пародия, Самураи...",3742.0,199.0,20.0,Gintama: Enchousen,銀魂' 延長戦,7823.0,...,32.0,73.0,271.0,572.0,954.0,9.08,Великолепно,NaN,5.0,Sunrise
8,98.0,22 мин.,End Story 2nd Season,"Детектив, Комедия, Сверхъестественное",5873.0,167.0,23.0,Owarimonogatari Second Season,終物語,4029.0,...,10.0,43.0,119.0,360.0,580.0,9.06,Великолепно,NaN,5.0,SHAFT
9,218.0,1 час 50 мин.,Gintama: The Final Chapter - Be Forever Yorozu...,"Экшен, Фантастика, Комедия, Исторический, Паро...",3076.0,126.0,20.0,NaN,劇場版 銀魂 完結篇 万事屋よ永遠なれ,7442.0,...,42.0,74.0,246.0,566.0,860.0,9.05,Великолепно,NaN,4.0,Sunrise


In [31]:
#приведение "1 час 2 минуты" к формату "62 минуты"
def to_time(string):
    if pd.isnull(string):
        return np.nan
    keyword_processor = KeywordProcessor()
    keyword_processor.add_keyword('мин.', ' ')
    string = keyword_processor.replace_keywords(string).strip()
    if 'час' in string:
        string = string.split('час')
        string[0] = 60 * int(string[0])
        try:
            string[1] = int(reduce(lambda y1, y2: y1 + y2, re.findall('\d', string[1])))
        except TypeError:
            del string[1]
        finally:
            return sum(string)       
    else:
        return int(string)
    
data['Length of the episode'] = data['Length of the episode'].apply(to_time)

In [32]:
#замена сокращений месяцов на полное название
def replace_month(string):
    keyword_processor = KeywordProcessor()
    """
    keyword_processor = KeywordProcessor()
    keyword_dict = {
    '1': ['января', 'янв.'],
    '11': ['ноября', 'нояб.']}
    keyword_processor.add_keywords_from_dict(keyword_dict)
    keyword_processor.replace_keywords('С 8 января 1988 г. по 17 нояб. 1997 г.')
    #output: 'С 8 1 1988 г. по 17 11 1997 г.'
    """
    keyword_processor.add_keyword('янв.', 'января')       
    keyword_processor.add_keyword('фев.', 'февраля')
    keyword_processor.add_keyword('апр.', 'апреля')
    keyword_processor.add_keyword('авг.', 'августа')
    keyword_processor.add_keyword('сент.', 'сентября')
    keyword_processor.add_keyword('окт.', 'октября')
    keyword_processor.add_keyword('нояб.', 'ноября')
    keyword_processor.add_keyword('дек.', 'декабря')
    keyword_processor.add_keyword('дек.', 'декабря')
    string = re.sub('-', 'г. ', string)
    return keyword_processor.replace_keywords(string).strip()

#нахождение дат и составления из них матрицы
#день выхода - месяц выхода - год выхода - день окончания - месяц окончания - год окончания 
def dates(string):
    extractor = DatesExtractor()
    matches = extractor(replace_month(string))
    if len(matches) == 0 or (not isinstance(string, str)):
        return [None] * 6
    elif len(matches) == 1:
        return [matches[0].fact.day, matches[0].fact.month, matches[0].fact.year] * 2 
    else:
        return [matches[0].fact.day, matches[0].fact.month, matches[0].fact.year, 
                matches[1].fact.day, matches[1].fact.month, matches[1].fact.year]

        
        
data_time = pd.DataFrame(columns = ['day start', 'month start', 'year start', 
                                  'day end', 'month end', 'year end', ])
for i, row in enumerate(data['Exit time']):
    try:
        data_time.loc[i] = dates(row)
    except:
        data_time.loc[i] = dates('')

In [17]:
genre = {'Action': ['Экшен'],
 'Adventure': ['Приключения'],
 'Cars': ['Машины'],
 'Comedy': ['Комедия'],
 'Dementia': ['Безумие'],
 'Demons': ['Демоны'],
 'Drama': ['Драма'],
 'Ecchi': ['Этти'],
 'Fantasy': ['Фэнтези'],
 'Game': ['Игры'],
 'Harem': ['Гарем'],
 'Historical': ['Исторический'],
 'Horror': ['Ужасы'],
 'Josei': ['Дзёсей'],
 'Kids': ['Детское'],
 'Magic': ['Магия'],
 'Martial Arts': ['Боевые искусства'],
 'Mecha': ['Меха'],
 'Military': ['Военное'],
 'Music': ['Музыка'],
 'Mystery': ['Детектив'],
 'Parody': ['Пародия'],
 'Police': ['Полиция'],
 'Psychological': ['Психологическое'],
 'Romance': ['Романтика'],
 'Samurai': ['Самураи'],
 'School': ['Школа'],
 'Sci-Fi': ['Фантастика'],
 'Seinen': ['Сейнен'],
 'Shoujo': ['Сёдзе'],
 'Shoujo Ai': ['Сёдзе Ай'],
 'Shounen': ['Сёнен'],
 'Shounen Ai': ['Сёнен Ай'],
 'Slice of Life': ['Повседневность'],
 'Space': ['Космос'],
 'Sports': ['Спорт'],
 'Super Power': ['Супер сила'],
 'Supernatural': ['Сверхъестественное'],
 'Thriller': ['Триллер'],
 'Vampire': ['Вампиры']}

In [33]:
len(genre)

40

In [34]:
keyword_processor = KeywordProcessor()
keyword_processor.add_keywords_from_dict(genre)
keyword_processor.replace_keywords
data['Genre'].fillna('hollow', inplace = True)
data['Genre'] = data['Genre'].apply(keyword_processor.replace_keywords)

In [35]:
class A:
    def __init__(self, corpus):
        self.corpus = np.array(corpus)
    @property
    def get_unique(self):
        uniq = [np.array([word.strip() for word in doc.split(',')]) for  doc in self.corpus]
        return set(reduce(lambda x, y: np.hstack((x, y)), uniq))
    def _get_vector(self, text):
        words = dict([(w.strip(), 1) for w in text.split(',')])
        return words        
    def get_matrix(self):
        terms = pd.DataFrame(columns = [genre for genre in self.get_unique])
        for doc in self.corpus:
            terms = terms.append(self._get_vector(doc), ignore_index = True)
        return terms.fillna(0).drop('hollow', axis = 1)
            
            
        

ob = A(data['Genre']).get_matrix()

data_now = pd.concat([data.drop('Exit time', axis = 1), data_time, ob], axis = 1).drop('Genre', axis = 1)
data_now

,Quit,Length of the episode,Other titles,Scheduled,Delayed,Revised,In English,In Japanese,Viewed,Rating,...,Police,Josei,Parody,Thriller,Music,Horror,Super Power,Slice of Life,Shounen,Shoujo
0,1084.0,24.0,"Hagane no Renkinjutsushi: Fullmetal Alchemist,...",12431.0,1273.0,490.0,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,38153.0,R-17,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,128.0,106.0,NaN,15627.0,287.0,129.0,Your Name.,君の名は。,42336.0,PG-13,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,488.0,24.0,Gintama' (2015),4635.0,370.0,45.0,Gintama Season 4,銀魂°,6672.0,R-17,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,2026.0,24.0,シュタゲ,17379.0,2024.0,382.0,Steins;Gate,シュタインズ・ゲート,42760.0,PG-13,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,443.0,24.0,Gintama (2011),4422.0,315.0,46.0,Gintama Season 2,銀魂',8704.0,PG-13,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,726.0,23.0,HxH (2011),7148.0,794.0,269.0,Hunter x Hunter,HUNTER×HUNTER ハンターハンター,14949.0,PG-13,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
6,238.0,26.0,"LoGH, LotGH, Gin'eiden, GinEiDen, Heldensagen ...",3872.0,286.0,12.0,Legend of the Galactic Heroes,銀河英雄伝説,1065.0,R-17,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,294.0,24.0,"Gintama' (2012), Gintama' Overdrive, Kintama",3742.0,199.0,20.0,Gintama: Enchousen,銀魂' 延長戦,7823.0,PG-13,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,98.0,22.0,End Story 2nd Season,5873.0,167.0,23.0,Owarimonogatari Second Season,終物語,4029.0,R-17,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,218.0,110.0,Gintama: The Final Chapter - Be Forever Yorozu...,3076.0,126.0,20.0,NaN,劇場版 銀魂 完結篇 万事屋よ永遠なれ,7442.0,PG-13,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [43]:
data_now.to_csv('proces.csv', sep = '\t', encoding = 'utf-8')